In [1]:
import copy,json
from operator import itemgetter
from visualize import plot_3d_rectangle
%matplotlib qt  
from helper import*
occupied_vol = 0
number_boxes = 0
value = 0
result = []
sorted_boxes = []
truck_dimension = [589,238, 238]
L,W,H = truck_dimension
PP = [[0,0,0]]#,[0,W,0]]
placed =[]
retry = []
best_point = []
overlaped = False
lcheck,wcheck,hcheck = [False, False,False]
# Plot using matplotlib
x_ticks = 600
y_ticks = 250
z_ticks = 250
step_size =50
effective_dimention ={0:[0,1,2],1:[1,0,2],2:[0,2,1],
                        3:[1,2,0],4:[2,1,0], 5:[2,0,1]}
#* reading data
# with open('data.json', 'r') as f:
#     data = json.load(f)

# sorted_boxes.extend(sorted([tup for tup in data if tup[0][-1] in {1}],
#                             key=lambda x: (x[1],x[0][3]), reverse=True))
# sorted_boxes.extend(sorted([tup for tup in data if tup[0][-1] in {2}],
#                             key=lambda x:(x[1],x[0][3]), reverse=True))
# sorted_boxes.extend(sorted([tup for tup in data if tup[0][-1] in {3}],
#                             key=lambda x: (x[1],x[0][3]), reverse=True))
# #sorted_boxes = sorted(sorted_boxes, key=lambda x: (x[0][1],x[0][-1]))
# for box, r in sorted_boxes:
#     box_vol = box[3]
#     box_value = box[4]
#     if r == 0:
#         l, w, h = box[0:3]
#     elif r == 1:
#         w, l, h = box[0:3]
#     elif r == 2:
#         l, h, w = box[0:3]
#     elif r == 3:
#         w, h, l = box[0:3]
#     elif r == 4:
#         h, w, l = box[0:3]
#     else:
#         h, l, w = box[0:3]

#     l,w,h = [box[i] for i in effective_dimention[r]]
    
#     for pos in PP:
            
#         if pos == [0,0,0]:

#             placed.extend([pos+[l,w,h,box[-1]]])
#             if pos[0]+l < L:
#                 PP.extend([ [pos[0]+l,0,0]])
#             if pos[1]+w < W:
#                 PP.extend([[0,pos[1]+w,0]])
#             if pos[1]+h < H:
#                 PP.extend([[0,0,pos[2]+h]])
#             PP.remove(pos)
#             plot_3d_rectangle(placed, x_ticks, y_ticks, z_ticks, step_size,0)
#             best_point.extend([pos])
#             PP=sorted(PP,key=lambda point: (point[0], point[2], point[1]), reverse=False)
#             break
#         else: 
#             if pos[0]+l < L and pos[1]+w < W and pos[2]+h < H: continue
#             while len(placed):
                # for box1 in placed:
                #     neb_pp = [
                #         [box1[0]+box1[3],box1[1],box1[2]],
                #         [box1[0],box1[1]+box1[4],box1[2]],
                #         [box1[0],box1[1],box1[2]+box1[5]]
                #     ]
#                     if pos in neb_pp:
#                         c=check_overlap(box1[0:6],pos+box[0:3])
#                         if check_overlap(box1[0:6],pos+box[0:3]): continue
#                         else:
#                             if check_hanging(box1[0:6],pos+box[0:3]): continue
#                             else:
#                                 placed.append(pos + [l,w,h,box[-1]])
                                
                                # if pos[0]+l < L:
                                #     PP.extend([ [pos[0]+l,pos[1],pos[2]]])
                                # if pos[1]+w < W:
                                #     PP.extend([[pos[0],pos[1]+w,pos[2]]])
                                # if pos[2]+h < H:
                                #     PP.extend([[pos[0],pos[1],pos[2]+h]])    
                                #     best_point.extend([pos])
#                                 PP.remove(pos)
#                                 PP=sorted(PP,key=lambda point: (point[0], point[2], point[1]), reverse=False)
                                
#                                 break
#                 else:
#                     continue
#         plot_3d_rectangle(placed, x_ticks, y_ticks, z_ticks, step_size,0)    
#         break



In [2]:
from visualize import plot_3d_rectangle
ss=[[0, 0, 0, 55, 92, 81, 1],
[0, 92, 0, 81, 92, 55, 1],
[0, 184, 0, 81, 55, 92, 1],[0,92,55,92,81,55,1]]


# 


# for item in ss:
#     print(intersect(item,[0,173,55,92,81,55,1]))
#[0, 0, 0, 55, 92, 81, 1],
plot_3d_rectangle([#[0,0,0,L,W,H,0],
                   [0, 0, 0, 55, 92, 81, 1],[0, 92, 0, 81, 92, 55, 1],
                   [0,92,55,92,81,55,1],[0,92,110,130,81,55,1],
                   [90, 92, 0, 81, 92, 55, 3],[130, 92, 0,110,130,81,2]]
                  , x_ticks, y_ticks, z_ticks, step_size,0)



In [20]:

def get_vertices(rect):
        # Extract dimensions of the rectangle
    x, y, z, l, w, h,_ = rect

    # Calculate the coordinates of all vertices of the rectangle
    return {        
        "FBL":[x, y, z],  # Front bottom left
        "FBR": [x + l, y, z],  # Front bottom right
        "FTL":[x, y , z+h],  # Front top left
        "FTR":[x + l, y , z+h],  # Front top right
        "BBL":[x, y+w, z],  # Back bottom left
        "BBR":[x + l, y+w, z],  # Back bottom right
        "BTL":[x, y + w, z + h],  # Back top left
        "BTR":[x + l, y + w, z + h]  # Back top right
    }


# def get_unsupported_vertices(rectangles):
#     unsupported_vertices = []
#     projections = []
#     supported = False
#     for rect in rectangles:
#         rect1 = get_vertices(rect)
#         for i in rectangles:
#             if i==rect:continue   
#             rect2 = get_vertices(i)
#             w=rect[2]+rect[-2]
#             if rect[2]+rect[-2] == i[2]:
#                 if i[0]+i[3] > rect[0]+rect[3]:
#                     unsupported_vertices.append((rect2["FBR"],rect2["BBR"]))
#                     projections.append(([rect2["FBR"][0],rect2["FBR"][1],abs(rect[4]-rect2["FBR"][2])],
#                                         [rect2["BBR"][0],rect2["BBR"][1],abs(rect[4]-rect2["BBR"][2])]))
#                 elif  i[1]+i[4] > rect[1]+rect[4]:
#                     unsupported_vertices.extend(rect2["BBL"] )  
#                     projections.append(([rect2["BBL"][0],rect2["BBL"][1],abs(rect[4]-rect2["BBL"][2])]))
    

#     return unsupported_vertices,projections

def get_projections(rectangles):
    projections = []
    for rect_i in rectangles:
        xi,yi,zi,li,wi,zi,c_flag = rect_i
        for rect_j in rectangles:
            if rect_i==rect_j: continue
            xj,yj,zj,lj,wj,zj,c_flag = rect_j
            if xi+li < xj+lj:
                vertices = get_vertices(rect_j)
                vertices["FBR"][2]=0
                #vertices["BBR"][2]=0
                
                projections.append(vertices["FBR"])
                #projections.append(vertices["BBR"])
            if yi+wi < yj+wj:
                vertices = get_vertices(rect_j)
                vertices["BBL"][2]=0
                #vertices["BBR"][2]=0
                projections.append(vertices["BBL"])
    return projections

# Example usage
ss = [
    [0,92,55,92,81,55,1],[0,92,110,130,81,55,1]
    # [0, 92, 0, 81, 92, 55, 1],
    # [0, 92, 55, 92, 81, 55, 1],
    # [0, 92, 110, 100, 81, 55, 1]
]

projections = get_projections(ss)
print(projections)
# Calculate unsupported vertices of ss[1] (on top of ss[0])
# unsupported_vertices_ss1 ,projections= get_unsupported_vertices(ss)

# print(unsupported_vertices_ss1)
# print(projections)


[[130, 92, 0]]


In [ ]:

import copy,json
from operator import itemgetter
from visualize import plot_3d_rectangle
%matplotlib qt  
from helper import*
occupied_vol = 0
number_boxes = 0
value = 0
result = []
sorted_boxes = []
truck_dimension = [589.8,243.8, 259.1]
VOL= truck_dimension[0]*truck_dimension[1]*truck_dimension[2]
L,W,H = truck_dimension
vol_items = 0
PP = [[0,0,0]]#,[0,W,0]]
removed_PP = []
placed =[]
retry = []
best_point = []
overlaped = False
lcheck,wcheck,hcheck = [False, False,False]
# Plot using matplotlib
x_ticks = 600
y_ticks = 250
z_ticks = 250
step_size =50
effective_dimention ={0:[0,1,2],1:[1,0,2],2:[0,2,1],
                        3:[1,2,0],4:[2,1,0], 5:[2,0,1]}



from box import  Item
from container import Container 
max_weight=28080
CONT = Container(
    name="ISO-20 feet Dry Container",
    LWH= (589.8,243.8,259.1),
    max_weight=28080,
)
print(CONT.string())
with open('data.json', 'r') as f:
    data = json.load(f)
boxes = [] 

for i in range(len(data)):
    box,r =data[i]
    boxes.append(Item(
        partno = f"Box-{i+1}",
        name= f"C-{box[-1]}",
        weight=box[3],
        LWH = box[0:3],
        rotation = r
    ))    
boxes=sorted(boxes,key=lambda item: (item.get_volume(),item.get_dimention()[0]),reverse=True)#,item[2],item[1]
boxes=sorted(boxes,key=lambda item: (item.get_id().split("C-")[1]),reverse=False)
L,W,H = CONT.get_dimention()

for box in boxes:

    l,w,h = box.get_dimention()

    for pos in CONT.PP:  
        if ( L < pos[0]+l or W < pos[1]+w or H < pos[2]+h ): continue
         
        if pos == [0,0,0]:

            box.set_onBase(True)
            box.set_position(pos)
            CONT.fit_items.append(box)
            # vol_items += box[4]
            temp_pp = []
            if pos[0]+l < L:
                temp_pp.append([pos[0]+l,pos[1],pos[2]])
            if pos[1]+w < W:
                temp_pp.append([pos[0],pos[1]+w,pos[2]])
            if pos[1]+h < H:
                temp_pp.append([pos[0],pos[1],pos[2]+h])
            CONT.PP.remove(pos)
            best_point.extend([pos])
            CONT.PP.extend(temp_pp)
            box.set_pps(temp_pp)
            CONT.PP=sorted(CONT.PP,key=lambda point: (point[0],point[2],point[1]), reverse=False)
            break   
        if ( L < pos[0]+l or W < pos[1]+w or H < pos[2]+h ): continue
        
        is_overlap = []
        support_area = []
        is_hanging = []
        temp = {}
        temp_pp = []
        for item in CONT.fit_items:
            box1 = item.get_position()+item.get_dimention()
            box2 = pos+box.get_dimention()
            is_overlap.append(intersect(box1,box2))
            if box1[5] == box2[2]:
                box.add_under(
                    [copy.deepcopy(item),calculate_overlap_area(box1, box2)])
                

        if True in is_overlap:
            # is_overlap = []
            # support_area =[]
            continue   
        if len(box.get_under()):
            under_sorted = sorted(box.get_under(), key=lambda x: x[1], reverse=True)
            if under_sorted[0][1] < 0.75:
                continue
            # print(box.get_under())
            # print(box.get_under()[0][1])
            # print(box.get_under().sort([0][1],reverse=True))
            # under_sorted = sorted(box.get_under(), key=lambda x: x[1], reverse=True)
            # if box.get_under().sort([0][1],reverse=True)<0.75: continue

        if pos[2] == 0:box.set_onBase(True)
        box.set_position(pos)
        CONT.fit_items.append(box)
        if pos[0]+l < L and [pos[0]+l,pos[1],pos[2]] not in CONT.PP:
           temp_pp.append( [pos[0]+l,pos[1],pos[2]])
        if pos[1]+w < W and [pos[0],pos[1]+w,pos[2]] not in CONT.PP:
            temp_pp.append([pos[0],pos[1]+w,pos[2]])
        if pos[2]+h < H and [pos[0],pos[1],pos[2]+h] not in CONT.PP:
            temp_pp.append([pos[0],pos[1],pos[2]+h])

        CONT.PP.remove(pos)
        best_point.extend([pos])
        CONT.PP.extend(temp_pp)
        box.set_pps(temp_pp)
        CONT.PP=sorted(CONT.PP,key=lambda point: (point[0],point[2],point[1]), reverse=False)
        
        break 
    
    
    
    
    bx = [box.get_plot_data() for box in CONT.fit_items]
    plot_3d_rectangle(bx, x_ticks, y_ticks, z_ticks, step_size,0)










# data2 = []
# for box,r in data:
    
#     l,w,h = [box[i] for i in effective_dimention[r]]
#     box[0:3] = l,w,h
#     data2.append(box)
# data2=sorted(data2,key=lambda dd: (dd[3],dd[0]),reverse=True)#,dd[2],dd[1]
# data2=sorted(data2,key=lambda dd: (dd[5]),reverse=False)

# print(len(data))
# with open('data2.json', 'w') as f:
#     json.dump(data2, f)

ISO-20 feet Dry Container, ((589.8, 243.8, 259.1), max_weight(28080),Vol(37256828.48)

KeyboardInterrupt - Ctrl-C again for new prompt


c:\Users\elahi\Desktop\AIDEAS_Local\CLP\visualize.py:98: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig = fig = plt.figure(figsize=(15,15))
